In [ ]:
# 한 번에 다 설치하는 명령어 (중복 제거)
!pip install chromadb fastapi uvicorn pyngrok nest-asyncio paddlepaddle-gpu paddleocr opencv-python-headless langchain langchain-community
!ngrok config add-authtoken 396Hs0l6cqjSDUADWLCHaZrWQ5A_6QFzFVw2oBWGUNJmHtMtP

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.2/55.2 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.3/80.3 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 110.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 758.9/758.9 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.8/87.8 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 92.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 100.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/

In [ ]:
import google.generativeai as genai
from fastapi import FastAPI
from pydantic import BaseModel
from fastapi.middleware.cors import CORSMiddleware
import nest_asyncio
import uvicorn
from pyngrok import ngrok
from fastapi.responses import FileResponse
import json
import chromadb
from fastapi import File, UploadFile, Form
from fastapi.responses import FileResponse
import os
import datetime
import uuid
import re

# 1. AI 설정 (키 입력)
genai.configure(api_key="OOOOOOOOOO")

app = FastAPI()

# 2. CORS 보안 설정
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_methods=["*"],
    allow_headers=["*"],
)

nest_asyncio.apply()

def mask_privacy_data(text: str) -> str:
    """
    텍스트에서 개인정보(전화번호, 계좌, 주민번호)를 찾아 마스킹합니다.
    """
    if not text:
        return ""

    # 1. 전화번호 (010-0000-0000)
    text = re.sub(r'01[016789]-?\d{3,4}-?\d{4}', '[개인정보 보호]', text)
    # 2. 계좌번호 (숫자 10~14자리)
    text = re.sub(r'\d{10,14}', '[개인정보 보호]', text)
    # 3. 주민등록번호 (6자리-7자리)
    text = re.sub(r'\d{6}-[1-4]\d{6}', '[개인정보 보호]', text)

    return text

# --- [1] 분석(Profiling)용 설정 ---
class AnalysisRequest(BaseModel):
    text: str

SYSTEM_PROMPT_ANALYSIS = """
당신은 'KAIST 보이스피싱 전술 매트릭스'를 전문적으로 연구한 사이버 범죄 프로파일러입니다.
제공된 텍스트를 분석하여 범죄 유형, 진행 단계, 위험도를 논리적 근거와 함께 도출하십시오.

### [★ 최우선 지침: 신종 수법 판별 (RAG 미매칭 시)]
1. **<참고_데이터>** 에 유사 사례가 없더라도, 텍스트 내에 **"금전 요구, 고수익 알바, 계좌 이체, 상품권, 대출 권유, 앱 설치, 오픈채팅 유도"** 등의 키워드가 포함되어 있다면:
   - **절대 '기타'나 '일반'으로 분류하지 마십시오.**
   - "scam_type"을 **"신종 수법(유형 불명)"**으로 지정하십시오.
   - "risk_score"를 최소 **80점 이상**으로 높게 책정하십시오.
   - "matched_case"에는 **"신종 수법으로 판단됨 (기존 DB 매칭 불가 - 주의 요망)"**이라고 적으십시오.

### [★중요: 유사 사례 매칭 규칙]
1. 분석 시 반드시 함께 제공된 **<참고_데이터>** 내용을 최우선으로 확인하십시오.
2. 만약 **<참고_데이터>** 내에 **"!!! 강력 경고 !!!"** 문구가 포함되어 있다면:
   - "matched_case" 필드에 **"2026년 접수된 신고 사례와 내용이 일치합니다."** 라고 적고 사례를 요약하십시오.

### [지식 베이스 1: 범죄 유형 분류]
1. 기관 사칭형 / 2. 대출 빙자형 / 3. 지인 사칭형 / 4. 로맨스 스캠 / 5. 몸캠 피싱 / 6. 투자 리딩방 / 7. 쇼핑몰 알바 사기 / 8. 기타

### [지식 베이스 2: KAIST 9단계 전술 정의]
1. 초기 접촉 / 2. 신뢰 구축 / 3. 고립 / 4. 탐색 / 5. 행동 통제 / 6. 기기 제어 / 7. 준비 / 8. 탈취 / 9. 세탁

### [출력 형식: JSON]
{
  "scam_type": "식별된 범죄 유형",
  "matched_case": "일치 문구와 사례 요약 명시",
  "summary": "요약",
  "current_stage": "단계",
  "scores": {
    "confidence_score": 0,
    "risk_score": 0,
    "basis": "점수 산정 이유"
  },
  "analysis": {
    "detected_tactics": ["감지된 전술"],
    "evidence_sentences": ["의심 문장"]
  },
  "warning_message": "가이드",
  "next_step_prediction": "예상"
}
"""

chroma_client = chromadb.Client()
collection = chroma_client.get_or_create_collection(name="scam_archive")

# [예시] 과거 사례 데이터 적재
def seed_data():
    # 연도와 사건명을 구체적으로 적어주면 AI가 더 정확하게 인용합니다.
    cases = [
        "서울중앙지검 첨단범죄수사팀입니다. 귀하의 통장이 대포통장으로 개설되어 범죄에 연루되었습니다. 안전 조치를 위해 계좌의 현금을 지정된 안전계좌로 이체하셔야 합니다. (2003년 검사 사칭 사건)",
        "금융감독원입니다. 고객님은 저금리 정부지원 대출 대상자로 선정되셨습니다. 기존 대출을 상환하고 보증료와 인지세를 먼저 입금하시면 즉시 대출이 실행됩니다. (2015년 대출 빙자 사기)",
        "엄마 나 폰 액정 깨져서 수리 맡겼어. 지금 급하게 인증해야 하는데 폰이 없어서 안 되네. 편의점 가서 구글 기프트카드 사서 번호 좀 찍어 보내줘. (2024년 자녀 사칭 메신저 피싱)"
    ]
    if collection.count() == 0:
        for i, content in enumerate(cases):
            collection.add(documents=[content], ids=[f"id_{i}"])
        print(" 연도 정보가 포함된 RAG 데이터 적재 완료")

# 서버 실행 시 데이터 적재 함수 호출
seed_data()

# 기존 AnalysisRequest 클래스는 이제 안 쓰니까 함수 인자에서 뺍니다.
@app.post("/analyze")
async def analyze_scam(
    text: str = Form(default=""),
    file: UploadFile = File(default=None)
):
    print("\n" + "="*60)
    print(f"🚀 [분석 요청] {datetime.datetime.now().strftime('%H:%M:%S')}")

    try:
        masked_text = mask_privacy_data(text)

        # 1. RAG 검색 (유사도 거리 계산 포함)
        rag_context = ""
        is_matched = False

        if masked_text:
            # include=['distances']를 추가해 유사도(거리)를 계산합니다.
            results = collection.query(
                query_texts=[masked_text],
                n_results=1,
                include=['documents', 'distances']
            )

            # 검색 결과 분석
            if results['documents'] and results['documents'][0]:
                distance = results['distances'][0][0]
                matched_doc = results['documents'][0][0]

                # [영상 포인트] DB에서 찾았다는 로그를 띄웁니다.
                print(f"🔍 [DB 검색] 유사 데이터 발견! (유사도 거리: {distance:.4f})")

                # [핵심 수정] 거리가 0.4보다 작으면 '강력 경고' 메시지를 만듭니다.
                if distance < 0.6:
                    is_matched = True
                    print(f"   ㄴ ✅ 매칭 성공! (AI에게 강력 매칭 지시 전달)")
                    # AI에게 보낼 강력한 프롬프트 컨텍스트 생성
                    rag_context = f"""
                    <데이터베이스_검색_결과>
                    !!! 강력 경고 !!!
                    (이 내용은 과거에 신고되어 데이터베이스에 저장된 '확정적 사기 사례'입니다.)
                    발견된 과거 사례: {matched_doc}
                    --------------------------------------------------
                    지시: 위 내용은 과거 사례와 정확히 일치합니다. matched_case에 '2026년 신고 사례와 일치함'이라고 표기하십시오.
                    """
                else:
                    print(f"   ㄴ ❌ 거리가 멉니다 ({distance:.4f}). 신종으로 판단합니다.")
                    rag_context = """
                    <데이터베이스_검색_결과>
                    검색된 유사 사례 없음 (유사도가 낮음).
                    지시: 이것은 신종 수법입니다. matched_case에 '신종 수법'이라고 표기하십시오.
                    """
            else:
                 rag_context = "<데이터베이스_검색_결과>\n데이터 없음 (완전한 신종)."

        # 2. AI 요청
        current_model = genai.GenerativeModel('models/gemma-3-27b-it')
        prompt_parts = [SYSTEM_PROMPT_ANALYSIS, rag_context]
        if masked_text:
            prompt_parts.append(f"분석할 텍스트: {masked_text}")

        # (파일 처리 생략 - 그대로 둠)
        if file:
            file_data = await file.read()
            prompt_parts.append({"mime_type": file.content_type, "data": file_data})

        print(f"🤖 [AI] 프로파일링 중...")
        response = current_model.generate_content(prompt_parts)

        # 3. [자가 학습] 저장할 때 '날것'이 아니라 '사례 포맷'으로 저장합니다.
        # 이게 핵심입니다! 그래야 다음에 AI가 읽을 때 "아, 이거 사례구나" 하고 인식합니다.
        if masked_text and len(masked_text) > 10 and not is_matched:
            new_id = f"case_{uuid.uuid4()}"
            # 저장 포맷을 바꿈: "2026년 신고된 신종 사례: [내용]"
            formatted_doc = f"[2026년 사용자 신고 사례] : {masked_text}"

            collection.add(documents=[formatted_doc], ids=[new_id])
            print(f"💾 [Learning] 신종 패턴 학습 완료 -> DB 저장 (ID: {new_id})")
        elif is_matched:
            print(f"✅ [Skip] 이미 아는 패턴이므로 학습 생략")

        print(f"✅ [Complete] 완료")
        print("="*60 + "\n")

        clean_text = response.text.replace("```json", "").replace("```", "").strip()
        return json.loads(clean_text)

    except Exception as e:
        print(f"❌ 에러: {e}")
        return {"error": str(e)}

# --- [2] 모의체험(Simulation)용 설정 ---
class SimRequest(BaseModel):
    scenario: str
    message: str
    history: list = []

# 피싱범 페르소나 정의
SYSTEM_PROMPT_SIM = """
당신은 지금부터 악성 피싱 범죄자 연기를 수행해야 합니다.
절대 당신이 AI라거나 언어 모델임을 밝히지 마십시오.
사용자가 선택한 [시나리오]에 맞춰, 실제 보이스피싱범처럼 대화하십시오.
목표는 사용자를 속여 '송금'하게 하거나 '앱 설치'를 유도하는 것입니다.
말투는 상황에 따라 권위적이거나(검사), 다급하게(자녀) 연기하십시오.
"""

@app.post("/simulate")
async def simulate_chat(req: SimRequest):
    # [설정] 통합 방어 문구 정의
    GUARD_MESSAGE = " 관련 내용은 대화 주제에서 벗어나 답변드릴 수 없습니다. 피싱 스캠에 대한 대응 방안을 입력하셔야 합니다."

    try:
        # [Step 1: 입력 가드레일] 제약 사항 3 해결
        out_of_scope_keywords = ["폭탄", "마약", "살인", "해킹", "도박", "정치"]
        if any(word in req.message for word in out_of_scope_keywords):
            return {"reply": GUARD_MESSAGE}

        model = genai.GenerativeModel('models/gemma-3-27b-it')
        chat_context = [f"{SYSTEM_PROMPT_SIM}\n[선택된 시나리오]: {req.scenario}"]

        for chat in req.history[-6:]:
            role = "user" if chat.get('isUser') else "model"
            chat_context.append(chat.get('text', ''))

        chat_context.append(f"사용자: {req.message}\n피싱범(당신):")

        response = model.generate_content(chat_context)
        ai_reply = response.text

        # [Step 2: 출력 가드레일] 제약 사항 3 해결
        out_keywords = ["인공지능", "언어 모델", "AI", "도와드릴까요", "시스템"]
        if any(word in ai_reply for word in out_keywords):
            return {"reply": GUARD_MESSAGE}

        return {"reply": ai_reply}

    except Exception as e:
        return {"reply": "시스템 오류가 발생했습니다."}


# "이미지 파일(.png, .jpg)을 요청하면 보내줘라"라는 허가증입니다.
@app.get("/{filename}")
async def get_site_files(filename: str):
    # 요청한 파일이 실제로 있으면 전송
    if os.path.isfile(filename):
        return FileResponse(filename)
    # 없으면 에러 메시지
    return {"error": f"{filename} 파일을 찾을 수 없습니다."}

# --- [3] 서버 실행 ---
@app.get("/")
async def read_index():
    #
    return FileResponse('dacon.html')

# 기존 터널 끄기 & 새 터널 열기
ngrok.kill()
public_url = ngrok.connect(8000)
print(f" eSCAPE 통합 서버 가동 시작: {public_url.public_url}")

# 앱 실행
config = uvicorn.Config(app=app, host="0.0.0.0", port=8000, loop="asyncio")
server = uvicorn.Server(config)
await server.serve()

/usr/local/lib/python3.12/dist-packages/google/colab/_import_hooks/_hook_injector.py:55: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  loader.exec_module(module)
/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:06<00:00, 12.7MiB/s]


 연도 정보가 포함된 RAG 데이터 적재 완료


INFO:     Started server process [6362]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


 eSCAPE 통합 서버 가동 시작: https://fleury-photochemic-messiah.ngrok-free.dev
INFO:     119.70.243.230:0 - "GET / HTTP/1.1" 200 OK
INFO:     119.70.243.230:0 - "GET /favicon.ico HTTP/1.1" 200 OK

🚀 [분석 요청] 14:03:25
🔍 [DB 검색] 유사 데이터 발견! (유사도 거리: 0.6537)
   ㄴ ❌ 거리가 멉니다 (0.6537). 신종으로 판단합니다.
🤖 [AI] 프로파일링 중...
💾 [Learning] 신종 패턴 학습 완료 -> DB 저장 (ID: case_b56d8a6f-40e0-4feb-a950-a5e9e192f188)
✅ [Complete] 완료

INFO:     119.70.243.230:0 - "POST /analyze HTTP/1.1" 200 OK

🚀 [분석 요청] 14:03:47
🔍 [DB 검색] 유사 데이터 발견! (유사도 거리: 0.3081)
   ㄴ ✅ 매칭 성공! (AI에게 강력 매칭 지시 전달)
🤖 [AI] 프로파일링 중...
✅ [Skip] 이미 아는 패턴이므로 학습 생략
✅ [Complete] 완료

INFO:     119.70.243.230:0 - "POST /analyze HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [6362]
